In [155]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import multiprocessing as mp
import math
import random
import sys
import time
import requests
import json
import numpy
import importlib
import Clases
from datetime import datetime,date,time, timedelta
from copy import deepcopy

In [156]:
r2 = requests.get(url='https://aviation-edge.com/v2/public/timetable?key=a24d93-2501aa&iataCode=LIM&type=arrival')
#http://aviation-edge.com/api/public/timetable?key=a24d93-2501aa&iataCode=LIM&type=arrival

In [157]:
data = r2.json()
print (data)

[{'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'AMS', 'icaoCode': 'EHAM', 'gate': 'E22', 'scheduledTime': '2019-05-07T12:35:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-07T18:05:00.000', 'estimatedTime': '2019-05-07T18:05:00.000'}, 'airline': {'name': 'Air France', 'iataCode': 'AF', 'icaoCode': 'AFR'}, 'flight': {'number': '8243', 'iataNumber': 'AF8243', 'icaoNumber': 'AFR8243'}, 'codeshared': {'airline': {'name': 'KLM', 'iataCode': 'KL', 'icaoCode': 'KLM'}, 'flight': {'number': '743', 'iataNumber': 'KL743', 'icaoNumber': 'KLM743'}}}, {'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'AMS', 'icaoCode': 'EHAM', 'gate': 'E22', 'scheduledTime': '2019-05-07T12:35:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-07T18:05:00.000', 'estimatedTime': '2019-05-07T18:05:00.000'}, 'airline': {'name': 'China Southern Airlines', 'iataCode': 'CZ', 'icaoCode': 'CSN'}, 'flight': {'n

In [158]:
data_filtered = list(filter(lambda x : x['status'] != 'landed' and x['status'] != 'cancelled', data))
print(data_filtered)

[{'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'AMS', 'icaoCode': 'EHAM', 'gate': 'E22', 'scheduledTime': '2019-05-07T12:35:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-07T18:05:00.000', 'estimatedTime': '2019-05-07T18:05:00.000'}, 'airline': {'name': 'Air France', 'iataCode': 'AF', 'icaoCode': 'AFR'}, 'flight': {'number': '8243', 'iataNumber': 'AF8243', 'icaoNumber': 'AFR8243'}, 'codeshared': {'airline': {'name': 'KLM', 'iataCode': 'KL', 'icaoCode': 'KLM'}, 'flight': {'number': '743', 'iataNumber': 'KL743', 'icaoNumber': 'KLM743'}}}, {'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'AMS', 'icaoCode': 'EHAM', 'gate': 'E22', 'scheduledTime': '2019-05-07T12:35:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-07T18:05:00.000', 'estimatedTime': '2019-05-07T18:05:00.000'}, 'airline': {'name': 'China Southern Airlines', 'iataCode': 'CZ', 'icaoCode': 'CSN'}, 'flight': {'n

In [179]:
listaVuelos = []
i =0
for flight in data_filtered:
    #print (flight['status'], flight['departure'], flight['arrival'], flight['airline'], flight['flight'])
    try:
        i +=1
        vuelo = Clases.Vuelo()
        jsonDestino = flight ['arrival']
        anho = int(jsonDestino['scheduledTime'][0:4])
        mes = int(jsonDestino['scheduledTime'][6:7])
        dia = int(jsonDestino['scheduledTime'][9:10])
        hora = int(jsonDestino['scheduledTime'][12:13])
        minuto = int(jsonDestino['scheduledTime'][15:16])
        segundo = int(jsonDestino['scheduledTime'][18:19])

        vuelo.addEstado(flight['status'])
        if (vuelo.estado=="scheduled"):
            vuelo.addTiempoProgramado(datetime(year=anho, month=mes, day=dia, \
                                       hour=hora, minute=minuto, second=segundo))
        elif (vuelo.estado=="active"):
            vuelo.addTiempoProgramado(datetime(year=anho, month=mes, day=dia, \
                                       hour=hora, minute=minuto, second=segundo))
            anho = int(jsonDestino['estimatedTime'][0:4])
            mes = int(jsonDestino['estimatedTime'][6:7])
            dia = int(jsonDestino['estimatedTime'][9:10])
            hora = int(jsonDestino['estimatedTime'][12:13])
            minuto = int(jsonDestino['estimatedTime'][15:16])
            segundo = int(jsonDestino['estimatedTime'][18:19])
            vuelo.addTiempoEstimado(datetime(year=anho, month=mes, day=dia, \
                                       hour=hora, minute=minuto, second=segundo))
        
        jsonVuelo = flight['flight']
        vuelo.addNumeroVuelo(jsonVuelo['number'])
        vuelo.addIata(jsonVuelo['iataNumber'])

        try:
            vuelo.addIcao(jsonVuelo['icaoNumber'])
        except:
            pass

        jsonAerolinea = flight['airline']
        aerolinea =Clases.TAerolinea()
        aerolinea.addIata(jsonAerolinea['iataCode'])
        try:
            aerolinea.addIcao(jsonAerolinea['icaoCode'])
        except:
            pass
        aerolinea.addNombre(jsonAerolinea['name'])

        avion = Clases.Avion()
        avion.addTAerolinea(aerolinea)
        vuelo.addAvion(avion)

        listaVuelos.append(vuelo)
        #print(i)
    except:
        pass

In [178]:
importlib.reload(Clases)

<module 'Clases' from 'C:\\Users\\jorge\\Documents\\2019-1\\AlgoRitmoAgogo\\AlgoRitmoAgogo\\Clases.py'>

In [180]:
nPuertas = 20
nZonas = 52
listaZonas = []
listaPuertas = []
for i in range(1,nPuertas+1):
    area2 = Clases.Puerta(2*i,random.random()*79+1, random.random()*59+1,random.random()*499+1,random.random()*499+1,10)
    listaPuertas.append(area2)
    
for i in range(1,nZonas +1):
    area = Clases.Zona(2*i-1,random.random()*79+1, random.random()*59+1,random.random()*499+1, random.random()*499+1)
    listaZonas.append(area)

In [181]:
print("cantVuelos: "+str(len(listaVuelos)))
ann = Annealer()

cantVuelos: 231
1 2019-05-07 08:05:00 2
2 2019-05-07 08:05:00 4
3 2019-05-07 08:05:00 6
4 2019-05-07 08:05:00 8
5 2019-05-07 08:05:00 10
6 2019-05-07 08:05:00 12
7 2019-05-07 08:05:00 14
8 2019-05-07 08:05:00 16
9 2019-05-07 08:04:00 18
10 2019-05-07 08:09:00 20
11 2019-05-07 08:03:00 22
12 2019-05-07 08:03:00 24
13 2019-05-07 08:03:00 26
14 2019-05-07 08:03:00 28
15 2019-05-07 08:00:00 30
16 2019-05-07 08:04:00 32
17 2019-05-07 08:04:00 34
18 2019-05-07 08:00:00 36
19 2019-05-07 08:08:00 38
20 2019-05-07 08:08:00 40
21 2019-05-07 08:08:00 1
22 2019-05-07 08:08:00 3
23 2019-05-07 08:00:00 5
24 2019-05-07 09:00:00 7
25 2019-05-07 09:00:00 9
26 2019-05-07 09:08:00 11
27 2019-05-07 09:08:00 13
28 2019-05-07 09:00:00 15
29 2019-05-07 09:00:00 17
30 2019-05-07 09:02:00 19
31 2019-05-07 09:02:00 21
32 2019-05-07 09:04:00 23
33 2019-05-07 09:05:00 25
34 2019-05-07 09:05:00 27
35 2019-05-07 09:05:00 29
36 2019-05-07 09:09:00 31
37 2019-05-07 09:09:00 33
38 2019-05-07 09:09:00 35
39 2019-05-07 

In [182]:
x,y = ann.anneal()
print("Resultado: " + str(y))

costo actual: 1177.9944444444445
costo actual: 1177.9944444444445
costo actual: 1177.9944444444445
costo actual: 1186.9944444444445
costo actual: 1192.9944444444445
costo actual: 1192.9944444444445
costo actual: 1192.9944444444445
costo actual: 1198.9944444444445
costo actual: 1198.9944444444445
costo actual: 1198.9944444444445
costo actual: 1198.9944444444445
costo actual: 1204.9944444444445
costo actual: 1201.9944444444445
costo actual: 1204.9944444444445
costo actual: 1201.9944444444445
costo actual: 1204.9944444444445
costo actual: 1201.9944444444445
costo actual: 1207.9944444444445
costo actual: 1204.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actu

costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1228.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1225.9944444444445
costo actual: 1225.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1225.9944444444445
costo actu

costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actu

costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1219.9944444444445
costo actual: 1216.9944444444445
costo actual: 1216.9944444444445
costo actual: 1219.9944444444445
costo actual: 1222.9944444444445
costo actual: 1222.9944444444445
costo actu

costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1216.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actu

...

costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actu

costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actu

costo actual: 1201.9944444444445
costo actual: 1204.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1204.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1201.9944444444445
costo actual: 1204.9944444444445
costo actual: 1207.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1204.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actu

costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1216.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1213.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1210.9944444444445
costo actual: 1207.9944444444445
costo actual: 1207.9944444444445
costo actual: 1210.9944444444445
costo actu

(([<Clases.Puerta at 0x153a4809630>,
   <Clases.Zona at 0x153a4a88048>]),
 4240780.0)

In [183]:
class Annealer(object):
    """
    Clase annealer
    """
    # parámetros
    Tmax = 25000.0
    Tmin = 2.5
    steps = 5000
    updates = 100
    user_exit = False
    max_iter = 1000

    best_state = None
    best_energy = None
    start = None

    def __init__(self):
        """
        Método FIFO para alcanzar una solución mas o menos óptima
        """
        j=0
        for vuelo in listaVuelos:
            asignado = False
            j+=1
            for puerta in listaPuertas:
                if (puerta.insertarVuelo(vuelo)!=-1):
                    asignado = True7
                    printVueloArea(j,vuelo,puerta)
                    break
                    
            if (asignado):
                continue
                
            for zona in listaZonas:
                if (zona.insertarVuelo(vuelo)!=-1):
                    asignado = True
                    printVueloArea(j,vuelo,zona)
                    break

            while(not asignado):
                if (vuelo.estado == "scheduled"):
                    print (j,vuelo.tiempoProgramado, "GG")
                    vuelo.tiempoProgramado += timedelta(hours=1)
                else:
                    print(j,vuelo.tiempoEstimado, "GG")
                    vuelo.tiempoEstimado += timedelta(hours=1)
                
                for puerta in listaPuertas:
                    if (puerta.insertarVuelo(vuelo)!=-1):
                        asignado = True
                        if (vuelo.estado == "scheduled"):
                            vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                        else:
                            vuelo.tiempoEstimado= vuelo.tiempoSinModificar                            
                        printVueloArea(j,vuelo,puerta)
                        break

                if (asignado):
                    asignado = True
                    break

                for zona in listaZonas:
                    if (zona.insertarVuelo(vuelo)!=-1):
                        asignado = True
                        if (vuelo.estado == "scheduled"):
                            vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                        else:
                            vuelo.tiempoEstimado= vuelo.tiempoSinModificar 
                        printVueloArea(j,vuelo,zona)
                        break
        self.state = (listaPuertas, listaZonas)
        best_state = deepcopy(self.state)
        best_energy = self.energy()

    def move(self):
        """Create a state change"""
        selector =round(random.random()*2)
        #if (selector == 0 ):
        #mov insercion 
        indiceArea = round(random.random())
        itera =0
        while (True):
            itera +=1
            if(itera >= 1000):
                self.move()
                break

            if(indiceArea == 0):
                indicePuertas = round(random.random()*(nPuertas-1))
                #print(indicePuertas)
                area = listaPuertas[indicePuertas]
            else:
                indiceZonas = round(random.random()*(nZonas-1))
                #print(indiceZonas)
                area = listaZonas[indiceZonas]

            if(area.vuelos.cantidad == 0):
                continue
            indiceVuelo = round(random.random()*area.vuelos.cantidad)+1
            cont = 1
            p = area.vuelos.inicio
            while(p is not None):
                if (p.ocupado and cont == indiceVuelo):
                    area.removeVuelo(p)
                    #print (p.vuelo.printData())
                    for puerta in listaPuertas:
                        if (puerta != area and puerta.insertarVuelo(p.vuelo)!=-1):
                            return

                    for zona in listaZonas:
                        if (zona != area and zona.insertarVuelo(p.vuelo)!=-1):
                            return  

                cont +=1                              
                p=p.sig

        #elif(selector == 1):
        #    pass
        #else:
        #    pass

    def energy(self):
        """Calculate state's energy"""
        costo = 0
        for puerta in listaPuertas:
            costo += puerta.vuelos.tiempoLibre.total_seconds()   
        
        for zona in listaPuertas:
            costo += zona.vuelos.tiempoLibre.total_seconds()   
        
        print("costo actual: "+ str(costo/3600) + " horas no usadas ")
        return costo
        
    def set_user_exit(self, signum, frame):
        """Raises the user_exit flag, further iterations are stopped
        """
        self.user_exit = True

    def default_update(self, step, T, E, acceptance, improvement):
        pass
        #elapsed = now() - self.start
        ''' 
        if step == 0:
            print(' Temperature        Energy    Accept   Improve     Elapsed   Remaining',
                  file=sys.stderr)
            print('\r%12.5f  %12.2f                      %s            ' %
                  (T, E, "", file=sys.stderr, end="\r") 
                  #time_string(elapsed))
            sys.stderr.flush()
        else:
            #remain = (self.steps - step) * (elapsed / step)
            print('\r%12.5f  %12.2f  %7.2f%%  %7.2f%%  %s  %s\r' %
                  (T, E, 100.0 * acceptance, 100.0 * improvement,
                   0,0, file=sys.stderr, end="\r") 
                  #time_string(elapsed), time_string(remain))
            sys.stderr.flush()
        '''
    def anneal(self):
        """
        Minimizar el tiempo de espera de todos los vuelos ya asignados (tiempo real - tiempo programado) 
        y el tiempo sin usar de las puertas del aeropuerto.
        Parameters
        state : an initial arrangement of the system
        Returns
        (state, energy): the best state and energy found.
        """
        step = 0
        #self.start = now()
        # Precompute factor for exponential cooling from Tmax to Tmin
        if self.Tmin <= 0.0:
            raise Exception('Exponential cooling requires a minimum "\
                "temperature greater than zero.')
        Tfactor = -math.log(self.Tmax / self.Tmin)

        # Note initial state
        T = self.Tmax
        E = self.energy()
        prevState = deepcopy(self.state)
        prevEnergy = E
        self.best_state = deepcopy(self.state)
        self.best_energy = E
        trials, accepts, improves = 0, 0, 0
        if self.updates > 0:
            updateWavelength = self.steps / self.updates
            self.default_update(step, T, E, None, None)

        # Attempt moves to new states
        while step < self.steps and not self.user_exit:
            step += 1
            T = self.Tmax * math.exp(Tfactor * step / self.steps)
            self.move()
            E = self.energy()
            dE = E - prevEnergy
            trials += 1
            if dE > 0.0 and math.exp(-dE / T) < random.random():
                # Restore previous state
                self.state = deepcopy(prevState)
                E = prevEnergy
            else:
                # Accept new state and compare to best state
                accepts += 1
                if dE < 0.0:
                    improves += 1
                prevState = deepcopy(self.state)
                prevEnergy = E
                if E < self.best_energy:
                    self.best_state = deepcopy(self.state)
                    self.best_energy = E
            if self.updates > 1:
                if (step // updateWavelength) > ((step - 1) // updateWavelength):
                    self.default_update(
                        step, T, E, accepts / trials, improves / trials)
                    trials, accepts, improves = 0, 0, 0

        # Return best state and energy
        return self.best_state, self.best_energy

In [154]:
def round_figures(x, n):
    """Returns x rounded to n significant figures."""
    return round(x, int(n - math.ceil(math.log10(abs(x)))))

def time_string(seconds):
    """Returns time in seconds as a string formatted HHHH:MM:SS."""
    s = int(round(seconds))  # round to nearest second
    h, s = divmod(s, 3600)   # get hours and remainder
    m, s = divmod(s, 60)     # split remainder into minutes and seconds
    return '%4i:%02i:%02i' % (h, m, s)

In [153]:
def printVueloArea(j,vuelo,puerta):
    
    if (vuelo.estado == "scheduled"):
        print (j,vuelo.tiempoProgramado, puerta.idArea)
    else:
        print(j,vuelo.tiempoEstimado, puerta.idArea)